# Introduction

There was strong objection to viability of quantum computers due to difficulty of noise suppression. [There still is some ](https://spectrum.ieee.org/computing/hardware/the-case-against-quantum-computing?fbclid=IwAR3diA9YlQXUUQKq_nfN1-2jj7pk25HkLTBI2YJDBY5SbF8xeLZlxY8MIS8).  
In an early paper [Physical Review A 51, 992 (1995)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.51.992) W. G. Unruh found that the coupling to the environment sets an ultimate time and size limit for any quantum computation. This initially curbed the hopes that the full advantage of quantum computing could be harnessed, since it set limits on the scalability of any algorithm. This problem was, at least in theory, remedied with the advent of quantum error correction, [P. W. Shor, Physical review A 52, R2493 (1995)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.R2493). It was proven that if both the decoherence and the imprecision of gates could be reduced below a finite threshold value, then quantum computation could be performed indefinitely [A. Y. Kitaev, Russian Mathematical Surveys 52, 1191 (1997)](https://iopscience.iop.org/article/10.1070/RM1997v052n06ABEH002155/meta). Although it is the ultimate goal to reach this threshold in an experiment that is scalable to larger sizes, the overhead that is needed to implement a fully fault-tolerant gate set with current codes [A. G. Fowler, et. al., Physical Review A 86, 032324 (2012)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.86.032324) seems prohibitively large [N. C. Jones, et. al., Physical Review X 2,
031007 (2012)](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.2.031007). In turn, it is expected that in the near term the progress in quantum experiments will lead to devices with dynamics, which are beyond what can be simulated with a conventional computer. This leads to the question: what computational tasks could be accomplished with only limited, or no error correction? 

The suggestions of near-term applications in such quantum devices mostly center around quantum simulations with short-depth circuit [D. Wecker, et. al., Physical Review A
92, 042303 (2015)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.042303) and approximate optimization algorithms [E. Farhi, et. al](https://arxiv.org/abs/1411.4028). Furthermore, certain problems in material simulation may be tackled by hybrid quantum-classical algorithms [Bela Bauer,et. al](https://arxiv.org/abs/1510.03859). In most such applications, the task can be abstracted to applying a short-depth quantum circuits to some simple initial state and then estimating the expectation value of some observable after the circuit has been applied. This estimation must be accurate enough to achieve a simulation precision comparable or exceeding that of classical algorithms. Yet, although the quantum system evolves coherently for the most part of the short-depth circuit, the effects of decoherence already become apparent as an error in the estimate of the observable. For the simulation to be of value, the effect of this error needs to be mitigated.

In the next sections we describe, implement and discuss DD, EM and their hybrid, respectively.

# Dynamical Decoupling
### Introduction
 In the NISQ era when the error correcting protocols are not feasible the role of error mitigating techniques rises. DD is one of such techniques from the error mitigation arsenal. It's a well-studied method which is designed to suppress the effects of decoherence via active intervention to the system evolution. Namely, by applying certain  control sequences on the system one can cancel the system-environment interaction to a given order. 
Here we discuss the theoretical framework of DD and then investigate it on quantum virtual machine on Rigetti platform. 
    
    
Some useful references. 
1. J. Preskill, [arXiv:1801.00862 (2018)](https://arxiv.org/abs/1801.00862)
1. B. Pokharel, et al., [arXiv:1807.08768v2 (2018)](https://arxiv.org/abs/1807.08768)
1. D. A. Lidar, [arXiv:1208.5791v3 (2013)](https://arxiv.org/abs/1208.5791)
1. L. Viola, E. Knill, and S. Lloyd, [Phys. Rev. Lett. **82**, 2417 (1999)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.2417)
1. L. Viola and S. Lloyd, [Phys. Rev. A **58**, 2733 (1998)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.58.2733)

### The pure dephasing
#### The ideal pulse case
Consider the Hamiltonian of a single-qubit system with pure dephasing system-bath coupling to the environment (bath): <p>

<center> $H = H_{s} + H_{sb} + H_{b}$, </center>

where $H_{s} = \lambda(t) \sigma^{x}$ is the system Hamiltonian, $H_{sb} = \sigma^{z} \otimes B^z$ is the coupling Hamiltonian and the last term is the bath Hamiltonian and for simplicity we set $H_{b} = 0$. We assume that $\lambda(t)$ is fully controlable field, for instance, several pulses of elctromagnetic field applied to the system. If these pulses last for a time $\delta$ with strength $\lambda$ then the following condition is satisfied $\delta \lambda = \frac{\pi}{2}$. <p>
Now let's proceed to the investigation of the evolution of system with following scenario: at the time $t = 0$ we turn on the pulse for a period of time $\delta$, then let the system interact for a period of time $\tau$, then repeat this procedure. Now we can write the corresponding unitary operators for the evolution of the system during the times $\delta$ and $\tau$: <p>
<center> $U_x = e^{-i\delta \lambda \sigma^x} \otimes I_b$, $U_f = e^{-i\tau H_{sb}}$. </center> <p>
In the case of an ideal pulse ( $\delta \rightarrow 0, \lambda \rightarrow \infty$) there is no system-bath interaction during the time the pulse is turned on. <p> 

Note that $U_x = e^{-i\delta \lambda \sigma^x} \otimes I_b = e^{-i \frac{\pi}{2} \sigma^x} \otimes I_b = - i \sigma^x \otimes I_b $. The joint system-bath evolution operator at time $ t = 2 \tau$ is (dropping overal factors)<p>
<center> $U_x U_f U_x U_f = (\sigma^x \otimes I_b)  e^{-i\tau H_{sb}} (\sigma^x \otimes I_b ) e^{-i\tau H_{sb}} = e^{-i\tau (\sigma^x \otimes I_b ) H_{sb} (\sigma^x \otimes I_b)} e^{-i\tau H_{sb}}= e^{-i\tau (\sigma^x \sigma^{z} \sigma^x) \otimes B^z } e^{-i\tau H_{sb}}$</center> <p>
where we used the identity $U e^A U^{\dagger} = e^{UAU^{\dagger}} $ (A is any operator and U is unitary). Pauli matrices anticommutes for each distinct pair and taking this into account we get 
<center> $ U_x U_f U_x U_f = e^{-i\tau (\sigma^x \sigma^{z} \sigma^x) \otimes B^z } e^{-i\tau H_{sb}}  = e^{i\tau \sigma^{z}  \otimes B^z } e^{-i\tau H_{sb}} = e^{i\tau H_{sb}} e^{-i\tau H_{sb}} = I$. </center> <p>
Thus, the effect of bath on the system eliminated at $t = 2 \tau$ and if we repeat the this procedure over and over the system will decouple from the bath every $2 \tau$.  
    
    
#### <center> The real pulse case </center>
In reality the pulse duration and strength don't satisfy the conditions for ideal case ($\delta \rightarrow 0, \lambda \rightarrow \infty$). Subsequently, during the interval when the pulse is applied the system-bath interaction cannot be neglected. If we take this into account and also the fact that indeed $H_b \neq 0$ then the evolution operators take the following form: <p>
<center> $U_x = e^{-i\delta ( \lambda \sigma^x + H_{sb} + H_b)}$, $U_f = e^{-i\tau (H_{sb}+H_b)}$ </center> <p>
Here we (see [3] for details) and directly write down the evolution operator at $t = 2(\tau + \delta)$ <p>
<center> $U_x U_f U_x U_f \equiv U_{2\tau } = I_s \otimes e^{-2i(\tau + \delta)H_b} + O\left((\tau +\delta)^2 ( \Vert H_{sb}\Vert + \Vert H_b\Vert)^2 \right) + O\left(\delta ( \Vert H_{sb}\Vert + \Vert H_b\Vert) \right) $</center>
where $\Vert A \Vert = \underset{\vert\psi \rangle}{sup} \equiv \frac{\Vert A \vert \psi \rangle \Vert}{\Vert \vert \psi \rangle \Vert} $  is the operator norm. <p> 
Now we get a condition on the pulse duration which will assure us that even in this real case we will gain the desired effect:   
<center> $\delta \ll \tau \ll \frac{1}{\Vert B^z \Vert + \Vert H_b \Vert}$. </center>    


### General Decoherence
Having considered the simplest coupling let's move to the general one-qubit system-bath coupling Hamiltonian <p>
<center> $H_{sb} = \sum \limits_{\alpha = x,y,z} \sigma^{\alpha} \otimes B^{\alpha}$ </center> <p>
Using the anticommutation relations of Pauli operators, we get <p>
<center> $\sigma^x H_{sb} \sigma^x = \sigma^x \otimes B^x - \sigma^y\otimes B^y - \sigma^z \otimes B^z  $</center> <p>

subsequently we can cancel will cancel the y and z contributions $ U_{2\tau} \equiv U_x U_f U_x U_f  = e^{-2\tau(\sigma^x \otimes B^x + H_b)} + O(\tau^2) $, where again $ U_f = e^{-i\tau(H_{sb} + H_b)} $ and it's supposed that the pulse is ideal.There remains only the x term in $H_{sb}$. We can remove the remaining $\sigma^x \otimes B^x $ term by applying Y-type ($U_y = e^{-i\frac{\pi}{2} \sigma^y}$) sequence to $U_{2\tau}$: <p> 
<center> $U_yU_{2\tau} U_y U_{2\tau} = U_z U_f U_z U_f = e^{-i4 \tau H_b} + O(\tau^2)$  </center> <p>
This pulse sequence is considered as the universal decoupling sequence for a single qubit since it removes a general system-bath interaction.
    

### Dynamical decoupling vs free evolution on rigetti's quantum virtual machine (QVM)
In order to study the performance of DD on qvm we mainly follow the strategy outlined in [[2](https://arxiv.org/abs/1807.08768)]. 
We investigate the fidelity decay for different initial states with free evolution and with DD sequences for one and two qubit systems. 
To get an estimation of performance of DD we generate random unitaries (circuits) and run them on qvm with decoherence noise. We run circuits above with different number of DD and idle sequences to figure out how DD reduces the fidelity decay (Need to be improved.) 
For the two qubit system the same routine is used to get the probabilities of measuring $ \vert 00 \rangle$.   
Our main focus is the supression of decoherence, so we model only decoherence noise to make evident 
the effects. 

To start implementing let's start by importing all the needed objects and functions for our simulation
```python
from pyquil import get_qc
from pyquil.quil import Program
from pyquil.gates import *
from pyquil.noise import add_decoherence_noise
import numpy as np
from helpers import * 
```
helpers.py contains functions which generate random unitaries, DD and idle sequences, 
and more functions for error itigation (see below).

We run circuits with two DD sequences, namely XYXY and ZXZX  and corresponding idle sequences. 
Below is given the description of variables used.  
* `n_Us`: number of random unitaries to generate test. 
* `sequences`: list of numbers, i.e. DD and idle sequences to apply
* `depth`:  the depth of the random unitaries (number of unitaries)
*  `shots`:  number of runs for each circuit

In this loop we generate n_Us random unitaries and save them in the list  
``` python
u_circuits = []
for i in range(n_Us):
    u_circuits.append(one_q_circuit(0,depth))
```

Now we construct the circuits $U DD U^{\dagger}$ with corresponing number of sequences, then run and store the data collected.

``` python
result_DDzx = np.zeros((n_Us,len(sequences),shots))

for idx1, p in enumerate(u_circs):
    for idx2, s in enumerate(sequences):
        p1 = get_zx_DD_sequence(0,s)
        p0 = p + p1 + p.dagger()
        p0_noisy = add_decoherence_noise(p0,ro_fidelity=1)
        ro = p0_noisy.declare('ro','BIT',1)
        p0_noisy+= MEASURE(0,ro[0])
             
        for it in range(shots): 
            qc = get_qc('1q-qvm')
            result_DDzx[idx1][idx2][it] = qc.run(p0_noisy)[0][0]
```


The same can be done for XYXY sequence and respective idle sequences we just need to change `get_zx_DD_sequence` function to `get_xy_DD_sequence` then to `get_idle_sequence`.  

We have run the simulation with the below parameter values
``` python
n_Us = 20
depth = 10
sequences = [0,12,24,36,48,60]
shots = 8500
```
After processing the data one can plot the fidelity decay depending on the number of sequences. Here the fidelity is defined as the total number of measured ground states ('0'-s) devided by number of shots. 
*Note that the $ZXZX$ sequence contains 4 pulses, while the $XYXY$ sequence contains 6 pulses. This is due to the native gate set: these are the gates (microwave pulses) that are actually being performed on rigetti's hardware.*

![1qzx](figures/DD_figs/1q_zx_fidelity.png)
![1qxy](figures/DD_figs/1q_xy_fidelity.png)
<p>
As we can see our simulation shows that it's beneficial to apply both sequences rather letting the system to evolve freely. To compare two sequences let us plot the curves in the same figure.
![1qall](figures/DD_figs/1q_all_fidelity.png)
<p>
As can be seen, for the one qubit case the XYXY outperforms the ZXZX a bit. 

For the two qubit case we constructed random unitaries with 10 clock cycles (each cycle consists of one CZ gate and two single qubit gates applied) and applied DD sequences on both qubits. In this case the fidelity is the total number of measured '00'-s devided by number of shots. 
Below are depicted the plots for this case.

![2qzx](figures/DD_figs/2q_00_zx_fidelity.png)
![2qxy](figures/DD_figs/2q_00_xy_fidelity.png)
<p>
DD mitigates the decoherence in the the two qubits as well. 
Putting all together we got an interesting result. 
![2qall](figures/DD_figs/2q_00_all_fidelity.png)
<p>
That is ZXZX performs better than XYXY which is right the opposite for the one qubit case

### Now we turn to explain the error mitigation technique and show its performance on a quantum computer simulator. Then we combine the both methods to see whether one can have better results.

# Error mitigation for quantum circuits

## Mittigation is important! How to mittigate then?

We want to estimate the expectation value of some quantum observable $A$ with respect to an evolved state $\rho_{\lambda}(T)$ after time $T$ that is subject to noise characterized by the parameter $\lambda$ in the limit where $\lambda \rightarrow 0$. To achieve this, one can apply Richardson’s deferred approach to the limit to cancel increasingly higher orders of $\lambda$, [K. Temme, et. al. arXiv:1612.02058](https://arxiv.org/abs/1612.02058).
The expectation value of the observable $A$ is obtained from the final state $\rho_{\lambda}(T)$ as $E_K(\lambda) = tr[A\rho_{\lambda}(T)]$. The function $E_K(\lambda)$ can be expressed as a series in $\lambda$ where the contribution with $\lambda_0$ corresponds to the noise-free evolution. This
can be seen by transforming the evolution into the interaction frame with regard to some time-ependent hamiltonian $K(t)$ and expanding in the Born series. Starting from the noise-free expectation value $E^\star = tr[A\rho_0(T)]$, the expansion is given by
$E_K(\lambda) = E^\star + \sum_{k=1}^n a_k\lambda_k + R_{n+1}(\lambda, \mathcal{L}, T )$, where $\mathcal{L}$ models the noise (Lindblad operator). 
The estimate of $E^\star$ can be significantly improved by considering the approximation $\hat{E}_K^n(\lambda)$, which is written as the linear combination
$$\hat{E}_K^n(\lambda) = \sum_{j=0}^n \gamma_j\hat{E}_K(c_j\lambda), \hspace{3cm} (1)$$
where the coefficients $\gamma_j$  and $c_j$ satisfy linear system of equations
$$\sum_j \gamma_j = 1, \hspace{1cm}  \sum_j \gamma_j c_j^k = 0. \hspace{2cm} (2)$$
where k = 1, ..., n.

The following is the protocol for estimates at different $\lambda_j = c_j\lambda$: <p>
1. For $j = 0, . . . , n$:  <p>
 &nbsp;&nbsp;&nbsp;&nbsp;     (a) choose a rescaling coefficient $c_j > 1$ ($c_0 = 1$) and ρ0 with rescaled Hamiltonian for time $Tj = c_jT$ 
<p>
&nbsp;&nbsp;&nbsp;&nbsp;      (b) Estimate observable A to obtain $\hat{E}_K(c_j\lambda)$. <p>
2.  Solve equations (2) and compute $\hat{E}_K^n(\lambda)$ as in Eq. (1).

Several choices for progression of $c_j$ are common in the literature.
So we can run the same program with different gate times and extrapolate to infinitely small time.

To test how well our method works we could simulate noisy quantum computer and run randomly generated programs with different depths and plot the results of mitigated and orignial fidelities of the results.

The figure below shows how error mitigation works for different noise parameters and mitigated order. We have used rigetti's forest sdk to simulate a noisy quantum computer and run up to 100 gates on it and compute the fidelities for various cases. The improvement due to error mitigation is obvious.

<img src="figures/mitigation_figs/mitigation.png" >

# Combining the Error Mitigation and Dinamical Decoupling

### Can we do even better by combining the two methods?

Lets first compare how those two compare to each other
<img src="figures/mitigation_figs/dd_and_em.png">

This should be investigated further for other conditions such us varying the frequency of additional DD gates, the stategy of placing those DD gates, the strategy of generating random cirquits, etc.
For combined results we are still getting incosistent results and try to investigate whether those methods work better.

We have tested our results against decoherence noise but we are planning to test our results with other types of errors as well.